In [ ]:
import numpy as np
import trimesh
import pyrender
import matplotlib.pyplot as plt
import os

os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["LIBGL_ALWAYS_SOFTWARE"] = "1"

SHOW_Y_AXIS = True

In [ ]:
# --- Custom LookAt Matrix Function ---
def look_at_matrix(eye, target, up):
    """
    Create a camera pose matrix (camera-to-world) for PyRender
    """
    f = target - eye
    f = f / np.linalg.norm(f) # Forward vector (from eye to target)

    # If the forward vector is parallel to the up vector, adjust the up vector
    # to avoid numerical instability in the cross product.
    if np.isclose(np.linalg.norm(np.cross(f, up)), 0):
        # Pick a new 'up' vector that is not parallel to 'f'
        if np.isclose(np.abs(np.dot(f, np.array([0, 1, 0]))), 1):
            up = np.array([0, 0, 1]) # If f is vertical, use Z as up
        else:
            up = np.array([0, 1, 0]) # Otherwise, use Y as up

    s = np.cross(f, up)
    s = s / np.linalg.norm(s) # Right vector

    u = np.cross(s, f)
    u = u / np.linalg.norm(u) # Corrected Up vector

    # Construct the view matrix (world-to-camera)
    view_matrix = np.array([
        [s[0], s[1], s[2], -np.dot(s, eye)],
        [u[0], u[1], u[2], -np.dot(u, eye)],
        [-f[0], -f[1], -f[2], np.dot(f, eye)],  # Camera looks down -Z axis
        [0, 0, 0, 1]
    ])

    # PyRender expects a pose matrix (camera-to-world), so we invert the view matrix
    camera_pose = np.linalg.inv(view_matrix)
    
    return camera_pose

# Load the mesh
lod_3_mesh = trimesh.load("/home/jeans/win/aaaJAIST/resources/LOD_data_50/1/lod3.obj")
lod_3_mesh.apply_translation(-lod_3_mesh.centroid)
scene = pyrender.Scene.from_trimesh_scene(lod_3_mesh)

In [ ]:
# Calculate bounding box for camera positioning
bounds = lod_3_mesh.extents
center = lod_3_mesh.centroid # This will now be [0,0,0] because we centered the mesh
radius = np.max(bounds) /2

print(f"Mesh bounds: {bounds}")
print(f"Mesh center: {center}")
print(f"Radius: {radius}")

# Create a bounding box mesh
# trimesh.primitives.Box creates a box centered at the origin by default.
# Its extents are given by `bounds`.
if SHOW_Y_AXIS:
    bbox_mesh = trimesh.primitives.Box(extents=[1,100000,1])
else:
    bbox_mesh = trimesh.primitives.Box(extents=[0,0,0])
    
# Setup Pyrender scene and renderer
scene = pyrender.Scene.from_trimesh_scene(lod_3_mesh) # Add the main mesh
r = pyrender.OffscreenRenderer(400, 400)

# Add the bounding box to the scene
# You might want to make it wireframe for better visualization.
# For wireframe, we need to create a material and set it.
bbox_material = pyrender.Material(wireframe=True)
bbox_pyrender_mesh = pyrender.Mesh.from_trimesh(bbox_mesh, material=bbox_material)
scene.add(bbox_pyrender_mesh)

# Raymond lighting (as in your original code)
intensity = 3.0

key = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity)
key_pose = np.array([
    [ 0,  0,  1,  2],
    [ 0,  1,  0,  2],
    [ 1,  0,  0,  2],
    [ 0,  0,  0,  1],
])
scene.add(key, pose=key_pose)

fill = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity * 0.5)
fill_pose = np.array([
    [ 0,  0, -1, -2],
    [ 0,  1,  0,  1],
    [-1,  0,  0, -2],
    [ 0,  0,  0,  1],
])
scene.add(fill, pose=fill_pose)

back = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity * 0.3)
back_pose = np.array([
    [ 1,  0,  0, -2],
    [ 0,  0,  1, -2],
    [ 0,  1,  0,  2],
    [ 0,  0,  0,  1],
])
scene.add(back, pose=back_pose)

# --- Camera Setup and Rendering Loop ---
camera_distance_factor = 2.0
camera_node = None

# Test with just a few angles first to debug
for camera_angle_deg in range(0, 360, 10):  # Changed to 45 degree steps for testing
    for camera_elevation_deg in range(0, 19, 20): # This loop will only run for elevation=0
        camera_angle_rad = np.deg2rad(camera_angle_deg)
        camera_elevation_rad = np.deg2rad(camera_elevation_deg)

        # Calculate camera position in spherical coordinates
        x = radius * camera_distance_factor * np.cos(camera_elevation_rad) * np.sin(camera_angle_rad)
        y = radius * camera_distance_factor * np.sin(camera_elevation_rad)
        z = radius * camera_distance_factor * np.cos(camera_elevation_rad) * np.cos(camera_angle_rad)

        eye = np.array([x, y, z])  # Camera position
        target = np.array([0.0, 0.0, 0.0])  # Look at origin (where we centered the mesh)
        up = np.array([0.0, 1.0, 0.0])  # World up vector

        # print(f"Camera angle: {camera_angle_deg}°, elevation: {camera_elevation_deg}°")
        # print(f"Eye position: {eye}")
        # print(f"Target position: {target}")
        # print(f"Distance from eye to target: {np.linalg.norm(eye - target)}")

        camera_pose = look_at_matrix(eye, target, up)
        # print(f"Camera pose matrix:\n{camera_pose}")

        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
        
        if camera_node is not None:
            scene.remove_node(camera_node)
        
        camera_node = scene.add(camera, pose=camera_pose)

        color, _ = r.render(scene)

        plt.figure(figsize=(3, 3))
        plt.axis('off')
        plt.imshow(color)
        plt.title(f'Angle: {camera_angle_deg}°, Elevation: {camera_elevation_deg}°')
        plt.show()

r.delete()